In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
congestion = 1

SUPPLY_BID = 0
LOAD_ATTACK = 0

LOAD1 = 0
DA_LOAD2 = 75       # in MWs

RT_LOAD2 = 90 + LOAD_ATTACK # in MWs

## DAM Optimization:

Objective function:
$$
\min \quad 0.5 p^T A p + b^T p
$$

Power balance constraint:
$$
\quad \quad \quad \quad \quad 1^T p = 0 \quad : \lambda 
$$ 

Line constraints:
$$
\quad \quad \quad \quad \quad -c \leq S \Phi p \leq c \quad : \mu^-, \mu^+
$$

Generation limits:
$$
\quad \quad \quad \quad \quad p^{min} \leq p \leq p^{max}
$$

where the optimization variables are cleared energy of physical and virtual supplies and loads in the system, defined as:

$$
p \triangleq [x \quad y \quad v \quad w]^T
$$


In [3]:
import os
import copy
import cvxpy as cp
import pandas as pd

def dcopf(gen, branch, gencost, bus, supply_vb):
    
    # Define sets
    G = gen['id'].values
    N = bus['bus_i'].values

    # Define base MVA
    baseMVA = gen['mbase'].iloc[0]

    # Decision variables
    GEN = cp.Variable(   len(G) ) # Generation
    THETA = cp.Variable( len(N) )             # Voltage phase angle of bus
    FLOW = cp.Variable( (len(N), len(N)) )    # Flows between all pairs of nodes
    
    # Objective function
    objective = cp.Minimize( 0.5 * cp.sum(cp.multiply(gencost['x1'], cp.square(GEN))) + cp.sum(cp.multiply(gencost['y1'], GEN)) )
    #objective = cp.Minimize( cp.sum( cp.multiply(gencost['x1'], GEN) ) )

    # Define the constraints
    constraints = []

    # Slack bus reference angle constraint
    constraints.append(THETA[0] == 0)
    
    # Supply-demand balance constraints
    for i in N:
        gen_indices = gen.loc[gen['bus'] == i, 'id'].values
        
        print( f"i: {i}, gen_indices: {gen_indices}")
        print( f"gen_indices-1: {gen_indices-1}" ) 
        print( f"GEN[gen_indices-1]: {GEN[gen_indices-1]}" ) 
        print(" ")
        
        pd_value = bus.loc[bus['bus_i'] == i, 'pd'].values[0]           # demand in MW
        fbus_values = branch.loc[branch['fbus'] == i, 'tbus'].values
        fbus_values_list = list(fbus_values) # Convert generator expression to a list
        flow_vars = [ FLOW[i-1, j-1] for j in fbus_values_list ]        # List of CVXPY flow variables
        
        if i==2: # Virtual bidding at Bus-2
            constraints.append(
                cp.sum(GEN[gen_indices-1]) + supply_vb - (pd_value) == cp.sum(flow_vars)
            )
        else:
            constraints.append(
                cp.sum(GEN[gen_indices-1]) - pd_value == cp.sum(flow_vars)
            )
    
    # Max generation constraints
    for g in G: # 1, 2
        constraints.append( GEN[g-1] <= gen.loc[g-1, 'pmax'] ) # also in MWs -- G1-g[0] AND G3-g[2] only 
    
    # Flow constraints on each branch
    for l in branch.index:
        from_bus = branch.loc[l, 'fbus']
        to_bus = branch.loc[l, 'tbus']
        susceptance = branch.loc[l, 'sus']
        constraints.append( FLOW[from_bus-1, to_bus-1] == baseMVA * susceptance * (THETA[from_bus-1] - THETA[to_bus-1]) )
    
    # Max line flow constraints
    for l in branch.index:
        from_bus = branch.loc[l, 'fbus']
        to_bus = branch.loc[l, 'tbus']
        constraints.append(FLOW[from_bus-1, to_bus-1] <= branch.loc[l, 'ratea'])
    
    # Define the problem and solve
    problem = cp.Problem(objective, constraints)
    #problem.solve(solver=cp.OSQP) # is an open-source C library for solving convex quadratic programs
    #problem.solve(solver=cp.ECOS)  # is an open-source C library for solving convex second-order and exponential cone programs. 
    problem.solve(solver=cp.XPRESS ) 
    
    # Prepare the output data
    generation = pd.DataFrame({
        'id': gen['id'],
        'node': gen['bus'],
        'gen': GEN.value
    })
    
    angles = THETA.value
    
    flows = pd.DataFrame({
        'fbus': branch['fbus'],
        'tbus': branch['tbus'],
        'flow': baseMVA * branch['sus'] * (angles[branch['fbus']-1] - angles[branch['tbus']-1])
    })
    
    # Extract the prices (dual values of the balance constraints)
    prices = pd.DataFrame({
        'node': bus['bus_i'],
        'value': [constraint.dual_value for constraint in constraints[1:len(N)+1]]
    })
    
    # Return the solution and objective value
    return {
        'generation': generation.round(7),
        'angles': [round(angle, 7) for angle in angles],
        'flows': flows.round(5),
        'prices': prices.applymap(lambda x: '{:.5f}'.format(x)),
        'cost': '{:.5f}'.format(problem.value),
        'status': problem.status
    }

/opt/homebrew/Caskroom/miniforge/base/envs/PGOC/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def input_data(datadir):

    # Read the CSV files into dataframes
    gen = pd.read_csv(os.path.join(datadir, 'gen.csv'))
    gencost = pd.read_csv(os.path.join(datadir, 'gencost.csv'))
    branch = pd.read_csv(os.path.join(datadir, 'branch.csv'))
    bus = pd.read_csv(os.path.join(datadir, 'bus.csv'))

    # Rename all columns to lowercase
    gen.columns = gen.columns.str.lower()
    gencost.columns = gencost.columns.str.lower()
    branch.columns = branch.columns.str.lower()
    bus.columns = bus.columns.str.lower()

    # Create generator ids
    gen['id'] = range(1, len(gen) + 1)
    #gen['id'] = [1, 3]
    gencost['id'] = [1, 2]

    # Create line ids
    branch['id'] = range(1, len(branch) + 1)

    # Add set of rows for reverse direction with same parameters
    branch2 = copy.deepcopy(branch)
    branch2['fbus'], branch2['tbus'] = branch2['tbus'], branch2['fbus']
    branch2 = branch2[branch.columns]  # Reorder columns to match original branch dataframe
    branch = pd.concat([branch, branch2], ignore_index=True)

    # Calculate the susceptance of each line
    # Assuming reactance is much greater than resistance (x >> 0)
    # Treat susceptance as the reciprocal of reactance (x)
    branch['sus'] = 1 / branch['x']
    
    return bus, gen, gencost, branch

In [5]:
baseKV = 230 * 1e3  # 230 kV
mbase  = 100 * 1e6  # 100 MVA
zbase  = ( baseKV * baseKV ) / mbase
x = 0.1
xpu = x/zbase
sus = 1/xpu
print(f"impedance_pu: {xpu}, sus_pu: {sus}")

impedance_pu: 0.0001890359168241966, sus_pu: 5290.0


$ Z_{\text{base}} = \frac{{V_{\text{base}}^2}}{{S_{\text{base}}}}, X_{\text{pu}} = \frac{{X}}{{Z_{\text{base}}}} $

In [6]:
# import numpy as np

# datadir = 'opf_data'
# DAM_bus, DAM_gen, DAM_gencost, branch = input_data(datadir)

# G = DAM_gen['id'].values
# #G = np.array([1, 3])
# N = DAM_bus['bus_i'].values
# G

In [7]:
print("Day Ahead Market(DAM) Data:\n")

datadir = 'opf_data'
DAM_bus, DAM_gen, DAM_gencost, branch = input_data(datadir)

print("Updated DAM Bus Dataframe:")
DAM_bus["pd"] = [LOAD1, DA_LOAD2, 0]
print(DAM_bus)
print("")

print("Updated DAM_gen:")
DAM_gen.loc[1, "bus"] = 3 # generator to bus 3 
print(DAM_gen)
print("")

print("Updated DAM_gencost:")
DAM_gencost["x1"] = [0.3, 0.8]
DAM_gencost["y1"] = [  3,   8]
print(DAM_gencost)
print("")

print("Updated DAM_branch:")
branch['x'] = xpu   # pu via per unit calculations
branch['sus'] = 1 / branch['x']

if congestion:
    branch.loc[((branch['fbus'] == 1) & (branch['tbus'] == 2)) | ((branch['fbus'] == 2) & (branch['tbus'] == 1)), 'ratea'] = 47  # creating congestion

print(branch)

Day Ahead Market(DAM) Data:

Updated DAM Bus Dataframe:
   bus_i  type  pd     qd  gs  bs  area  vm  va  basekv  zone  vmax  vmin
0      1     2   0   0.00   0   0     1   1   0     230     1   1.1   0.9
1      2     2  75   0.00   0   0     1   1   0     230     1   1.1   0.9
2      3     1   0  98.61   0   0     1   1   0     230     1   1.1   0.9

Updated DAM_gen:
   bus   pg  qg   qmax   qmin  vg  mbase  status  pmax  pmin  ...  qc1min  \
0    1   40   0   30.0  -30.0   1    100       1  1000     0  ...       0   
1    3  170   0  127.5 -127.5   1    100       1  1000     0  ...       0   

   qc1max  qc2min  qc2max  ramp_agc  ramp_10  ramp_30  ramp_q  apf  id  
0       0       0       0         0        0        0       0    0   1  
1       0       0       0         0        0        0       0    0   2  

[2 rows x 22 columns]

Updated DAM_gencost:
   model  startup  shutdown  n   x1  y1  id
0      2        0         0  2  0.3   3   1
1      2        0         0  2  0.8   8   2

U

In [8]:
print("Day Ahead Market(DAM) Results:\n")

DAM = dcopf(DAM_gen, branch, DAM_gencost, DAM_bus, SUPPLY_BID)

for key, value in DAM.items():
    print(f"{key}:") 
    print(value)
    print()

Day Ahead Market(DAM) Results:

i: 1, gen_indices: [1]
gen_indices-1: [0]
GEN[gen_indices-1]: var1[0]
 
i: 2, gen_indices: []
gen_indices-1: []
GEN[gen_indices-1]: var1[]
 
i: 3, gen_indices: [2]
gen_indices-1: [1]
GEN[gen_indices-1]: var1[1]
 
Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/opt/homebrew/Caskroom/miniforge/base/envs/PGOC/lib/python3.9/site-packages/xpress/license/community-xpauth.xpr')
generation:
   id  node        gen
0   1     1  59.090909
1   2     3  15.909091

angles:
[0.0, -8.45e-05, -2.72e-05]

flows:
   fbus  tbus      flow
0     1     3  14.39394
1     1     2  44.69697
2     2     3 -30.30303
3     3     1 -14.39394
4     2     1 -44.69697
5     3     2  30.30303

prices:
      node      value
0  1.00000  -20.72727


<ipython-input-3-cd910b6aaf57>:102: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'prices': prices.applymap(lambda x: '{:.5f}'.format(x)),


## Real-Time Market Optimization

$$
\min \quad 0.5 z^T C z + d^T z
$$

Power balance constraint:
$$
1^T z + 1^T x = 1^T l \quad : \delta
$$

Line constraints:
$$
-c \leq S(\Psi x + \Theta z - \Omega l) \leq c \quad : \eta^-, \eta^+
$$

Generation limits:
$$
z^{min} \leq z \leq z^{max}
$$

LMPs:
$$
\mathbf{\sigma}
$$

In [9]:
def RTM_dcopf(gen, branch, gencost, bus, **kwargs):
    
    # Define sets
    G = gen['id'].values
    N = bus['bus_i'].values

    # Define base MVA
    baseMVA = gen['mbase'].iloc[0]

    # Decision variables
    GEN = cp.Variable(   len(G) )    # Generation
    THETA = cp.Variable( len(N) )             # Voltage phase angle of bus
    FLOW = cp.Variable( (len(N), len(N)) )    # Flows between all pairs of nodes

    # Objective function
    objective = cp.Minimize( 0.5 * cp.sum(cp.multiply(gencost['x1'], cp.square(GEN))) + cp.sum(cp.multiply(gencost['y1'], GEN)) )
    #objective = cp.Minimize( cp.sum( cp.multiply(gencost['x1'], GEN) ) )

    # Define the constraints
    constraints = []

    # Slack bus reference angle constraint
    constraints.append(THETA[0] == 0)

    # Supply-demand balance constraints
    for i in N:
        gen_indices = gen.loc[gen['bus'] == i, 'id'].values
        pd_value = bus.loc[bus['bus_i'] == i, 'pd'].values[0]
        fbus_values = branch.loc[branch['fbus'] == i, 'tbus'].values
        fbus_values_list = list(fbus_values)  # Convert generator expression to a list
        flow_vars = [ FLOW[i-1, j-1] for j in fbus_values_list ]  # List of CVXPY flow variables
        
        if i==1:
            constraints.append(
                cp.sum(GEN[gen_indices-1]) + kwargs.get('arg1') - pd_value == cp.sum(flow_vars)
        )
        elif i==3:
            constraints.append(
                cp.sum(GEN[gen_indices-1]) + kwargs.get('arg2') - pd_value == cp.sum(flow_vars)
            )
        else:
            constraints.append(
                cp.sum(GEN[gen_indices-1]) - pd_value == cp.sum(flow_vars)
            )     
            
    # Max generation constraints
    for g in G:
        constraints.append(GEN[g-1] <= gen.loc[g-1, 'pmax'])
        
    #constraints.append(GEN[0] >= kwargs.get('arg1') )
    #constraints.append(GEN[2] >= kwargs.get('arg2') )

    # Flow constraints on each branch
    for l in branch.index:
        from_bus = branch.loc[l, 'fbus']
        to_bus = branch.loc[l, 'tbus']
        susceptance = branch.loc[l, 'sus']
        constraints.append( FLOW[from_bus-1, to_bus-1] == baseMVA * susceptance * (THETA[from_bus-1] - THETA[to_bus-1]) )
    
    # Max line flow constraints
    for l in branch.index:
        from_bus = branch.loc[l, 'fbus']
        to_bus = branch.loc[l, 'tbus']
        constraints.append( FLOW[from_bus-1, to_bus-1] <= branch.loc[l, 'ratea'] )
    
    # Define the problem and solve
    problem = cp.Problem(objective, constraints)
    #problem.solve(solver=cp.OSQP) # is an open-source C library for solving convex quadratic programs
    #problem.solve(solver=cp.ECOS)  # is an open-source C library for solving convex second-order and exponential cone programs. 
    problem.solve(solver=cp.XPRESS ) 
     
    
    # Prepare the output data
    generation = pd.DataFrame({
        'id': gen['id'],
        'node': gen['bus'],
        'gen': GEN.value
    })
    
    angles = THETA.value
    
    flows = pd.DataFrame({
        'fbus': branch['fbus'],
        'tbus': branch['tbus'],
        'flow': baseMVA * branch['sus'] * (angles[branch['fbus']-1] - angles[branch['tbus']-1])
    })
    
    # Extract the prices (dual values of the balance constraints)
    prices = pd.DataFrame({
        'node': bus['bus_i'],
        'value': [constraint.dual_value for constraint in constraints[1:len(N)+1]]
    })
    
    # Return the solution and objective value
    return {
        'generation': generation.round(7),
        'angles': [round(angle, 7) for angle in angles],
        'flows': flows.round(5),
        'prices': prices.applymap(lambda x: '{:.5f}'.format(x)),
        'cost': '{:.5f}'.format(problem.value),
        'status': problem.status
    }

In [10]:
print("Real Time Market(RTM) Data:\n")

datadir = 'opf_data'
RTM_bus, RTM_gen, RTM_gencost, _ = input_data(datadir)

print("Updated Bus Dataframe:")
RTM_bus["pd"] = [LOAD1, RT_LOAD2, 0]
print(RTM_bus)
print("")

print("Updated RTM_gen:")
# Create a DataFrame with the new row data
new_row = pd.DataFrame({'bus': [3], 'pg': [100], 'qg': [0], 'qmax': [0], 'qmin': [0], 'vg': [0], 'mbase': [100], 'status': [1], 'pmax': [1000], 'pmin': [0], 'qc1max': [0], 'qc1min': [0], 'qc2max': [0], 'qc2min': [0], 'ramp_agc': [0], 'ramp_10': [0], 'ramp_30': [0], 'ramp_q': [0], 'apf': [0], 'id': [3]})
RTM_gen = pd.concat([RTM_gen, new_row], ignore_index=True) # Append the new row to RTM_gen
print(RTM_gen)
print("")

print("Updated RTM_gencost:")
last_row = RTM_gencost.iloc[-1] # Get the last row of the DataFrame
RTM_gencost = pd.concat([RTM_gencost, pd.DataFrame([last_row])], ignore_index=True) # Concatenate the original DataFrame with the duplicated row
RTM_gencost["x1"] = [1.8, 1.7, 0.1]
RTM_gencost["y1"] = [ 10,  5,   14]
RTM_gencost["id"] = [  1,  2,    3]
print(RTM_gencost)
print("")

print("Updated RTM_branch:")
#RTM_branch['x'] = 0.00189
#if congestion:
#    RTM_branch.loc[((RTM_branch['fbus'] == 1) & (RTM_branch['tbus'] == 2)) | ((RTM_branch['fbus'] == 2) & (RTM_branch['tbus'] == 1)), 'ratea'] = 47  # creating congestion
print(branch)

Real Time Market(RTM) Data:

Updated Bus Dataframe:
   bus_i  type  pd     qd  gs  bs  area  vm  va  basekv  zone  vmax  vmin
0      1     2   0   0.00   0   0     1   1   0     230     1   1.1   0.9
1      2     2  90   0.00   0   0     1   1   0     230     1   1.1   0.9
2      3     1   0  98.61   0   0     1   1   0     230     1   1.1   0.9

Updated RTM_gen:
   bus   pg  qg   qmax   qmin  vg  mbase  status  pmax  pmin  ...  qc1min  \
0    1   40   0   30.0  -30.0   1    100       1  1000     0  ...       0   
1    2  170   0  127.5 -127.5   1    100       1  1000     0  ...       0   
2    3  100   0    0.0    0.0   0    100       1  1000     0  ...       0   

   qc1max  qc2min  qc2max  ramp_agc  ramp_10  ramp_30  ramp_q  apf  id  
0       0       0       0         0        0        0       0    0   1  
1       0       0       0         0        0        0       0    0   2  
2       0       0       0         0        0        0       0    0   3  

[3 rows x 22 columns]

Updated R

In [11]:
print("Real Time Market(RTM):\n")

RTM = RTM_dcopf(RTM_gen, branch, RTM_gencost, RTM_bus, arg1=DAM['generation']["gen"][0], arg2=DAM['generation']["gen"][1])

for key, value in RTM.items():
    print(f"{key}:") 
    print(value)
    print()

Real Time Market(RTM):

generation:
   id  node       gen
0   1     1  0.160839
1   2     2  8.251748
2   3     3  6.587413

angles:
[0.0, -8.88e-05, -2.32e-05]

flows:
   fbus  tbus      flow
0     1     3  12.25175
1     1     2  47.00000
2     2     3 -34.74825
3     3     1 -12.25175
4     2     1 -47.00000
5     3     2  34.74825

prices:
      node      value
0  1.00000  -10.28951
1  2.00000  -19.02797
2  3.00000  -14.65874

cost:
195.16154

status:
optimal



<ipython-input-9-8e0f91e04fa5>:99: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  'prices': prices.applymap(lambda x: '{:.5f}'.format(x)),


In [12]:
RTM['prices']['value'] = pd.to_numeric(RTM['prices']['value'])
RTM['prices']['value'] = RTM['prices']['value'].apply(lambda x: -x)

DAM['prices']['value'] = pd.to_numeric(DAM['prices']['value'])
DAM['prices']['value'] = DAM['prices']['value'].apply(lambda x: -x)

count = 1
for dam_price, rtm_price in zip(DAM['prices']['value'], RTM['prices']['value']):
    print(f"Bus {count}: {dam_price}, {rtm_price}, {dam_price - rtm_price:.5f}")
    if count == 2:
        print(f"Bus-2 Profit: {(dam_price - rtm_price)*SUPPLY_BID - (LOAD_ATTACK)*rtm_price}")
    count += 1

Bus 1: 20.72727, 10.28951, 10.43776
Bus 2: 20.72727, 19.02797, 1.69930
Bus-2 Profit: 0.0
Bus 3: 20.72727, 14.65874, 6.06853
